In [23]:
import torch
from torch import nn
import lightnet as ln
from torchinfo import summary
from model import model_builder
import functools

In [3]:
# model = ln.models.Darknet19(1000)
# model.load('weights/darknet19_448.weights')

# # Save as PyTorch weight file (Not strictly necessary, but it is faster than darknet weight files)
# model.save('weights/darknet19_448.pt')

# # Converting Darknet19 weights to Yolo (This is the same as the darknet19_448.conv.23.weights from darknet)
# model.save('weights/yolo-pretrained_darknet.pt', remap=ln.models.YoloV2.remap_darknet19)

# # Load yolo weights (Requires `strict=False`, because not all layers have weights in this file)
# detection_model = ln.models.YoloV2(3)
# detection_model.load('weights/yolo-pretrained_darknet.pt', strict=False)

## Depth Only Network

In [15]:
yolov2 = ln.models.YoloV2(2)
yolov2.load('weights/yolo-pretrained_darknet.pt', strict=False)


#for i in detection_model.state_dict().keys():
#    print(i)

/home/gustavo/workstation/depth_estimation/codes/rgbd-yolov2/.venv/lib/python3.10/site-packages/lightnet/network/module/_lightnet.py:293: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental 

In [16]:
yolov2.backbone.module[0]

Conv2dBatchAct(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), LeakyReLU(negative_slope=0.1, inplace=True))

In [17]:
print("Original first layer:", yolov2.backbone.module[0])
original_conv = yolov2.backbone.module[0]

Original first layer: Conv2dBatchAct(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), LeakyReLU(negative_slope=0.1, inplace=True))


In [28]:
new_conv = ln.network.layer.Conv2dBatchAct(in_channels=1, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), activation=functools.partial(nn.modules.activation.LeakyReLU, negative_slope=0.1, inplace=True))

In [33]:
yolov2.backbone.module[0] = new_conv

In [37]:


for i in yolov2.state_dict().keys():
    print(i)

backbone.module.1_convbatch.layers.0.weight
backbone.module.1_convbatch.layers.1.weight
backbone.module.1_convbatch.layers.1.bias
backbone.module.1_convbatch.layers.1.running_mean
backbone.module.1_convbatch.layers.1.running_var
backbone.module.1_convbatch.layers.1.num_batches_tracked
backbone.module.3_convbatch.layers.0.weight
backbone.module.3_convbatch.layers.1.weight
backbone.module.3_convbatch.layers.1.bias
backbone.module.3_convbatch.layers.1.running_mean
backbone.module.3_convbatch.layers.1.running_var
backbone.module.3_convbatch.layers.1.num_batches_tracked
backbone.module.5_convbatch.layers.0.weight
backbone.module.5_convbatch.layers.1.weight
backbone.module.5_convbatch.layers.1.bias
backbone.module.5_convbatch.layers.1.running_mean
backbone.module.5_convbatch.layers.1.running_var
backbone.module.5_convbatch.layers.1.num_batches_tracked
backbone.module.6_convbatch.layers.0.weight
backbone.module.6_convbatch.layers.1.weight
backbone.module.6_convbatch.layers.1.bias
backbone.mod

In [36]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=yolov2, 
        input_size=(32, 1, 416, 416), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
YoloV2 (YoloV2)                               [32, 1, 416, 416]    [32, 35, 13, 13]     --                   True
├─FeatureExtractor (backbone)                 [32, 1, 416, 416]    [32, 1024, 13, 13]   --                   True
│    └─Sequential (module)                    [32, 1, 416, 416]    [32, 1024, 13, 13]   --                   True
│    │    └─Conv2dBatchAct (1_convbatch)      [32, 1, 416, 416]    [32, 32, 416, 416]   352                  True
│    │    └─MaxPool2d (2_max)                 [32, 32, 416, 416]   [32, 32, 208, 208]   --                   --
│    │    └─Conv2dBatchAct (3_convbatch)      [32, 32, 208, 208]   [32, 64, 208, 208]   18,560               True
│    │    └─MaxPool2d (4_max)                 [32, 64, 208, 208]   [32, 64, 104, 104]   --                   --
│    │    └─Conv2dBatchAct (5_convbatch)      [32, 64, 104, 104]   [32, 128, 104, 104] 

In [5]:
model_fusion = ln.models.YoloFusion(2, fuse_layer=7)

for i in model_fusion.state_dict().keys():
    print(i)

layers.0.1_convbatch_regular.layers.0.weight
layers.0.1_convbatch_regular.layers.1.weight
layers.0.1_convbatch_regular.layers.1.bias
layers.0.1_convbatch_regular.layers.1.running_mean
layers.0.1_convbatch_regular.layers.1.running_var
layers.0.1_convbatch_regular.layers.1.num_batches_tracked
layers.0.1_convbatch_fusion.layers.0.weight
layers.0.1_convbatch_fusion.layers.1.weight
layers.0.1_convbatch_fusion.layers.1.bias
layers.0.1_convbatch_fusion.layers.1.running_mean
layers.0.1_convbatch_fusion.layers.1.running_var
layers.0.1_convbatch_fusion.layers.1.num_batches_tracked
layers.1.regular.3_convbatch.layers.0.weight
layers.1.regular.3_convbatch.layers.1.weight
layers.1.regular.3_convbatch.layers.1.bias
layers.1.regular.3_convbatch.layers.1.running_mean
layers.1.regular.3_convbatch.layers.1.running_var
layers.1.regular.3_convbatch.layers.1.num_batches_tracked
layers.1.regular.5_convbatch.layers.0.weight
layers.1.regular.5_convbatch.layers.1.weight
layers.1.regular.5_convbatch.layers.1.bi

In [6]:
yolov2 = model_builder(3)

Modules not matching, performing partial update


In [ ]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 